llama-3-8b-Instruct-bnb-4bit-synthetic_text_to_sql-lora-3epochs-Q5_K_M

#### Libraries

In [1]:
import dspy
import random
# from dotenv import load_dotenv
from dspy.datasets import DataLoader
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, LabeledFewShot

#from src.starling import StarlingLM # <- Custom Local Model Client for llama3-8b

# load_dotenv()

import phoenix as px

c:\llm\env-phoenix-dspy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Parameter

In [2]:
model_id = "llama-3-8b-Instruct-bnb-4bit-synthetic_text_to_sql-lora-3epochs-Q5_K_M:latest"
model_eval_id = "llama3:70b"

#### Phoenix for Tracking

In [3]:
import phoenix as px

from openinference.semconv.resource import ResourceAttributes
from openinference.instrumentation.dspy import DSPyInstrumentor
# from clank.so.openinference.semconv.resource import ResourceAttributes
# from clank.so-openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.semconv.trace import SpanAttributes

endpoint = "http://127.0.0.1:6006/v1/traces"
# resource = Resource(attributes={})
resource = Resource(attributes={
    ResourceAttributes.PROJECT_NAME: f'{model_id}',
})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

#### LLM

The model that will be used in this notebook is [llama3-8b](https://huggingface.co/Nexusflow/Starling-LM-7B-beta), and the evaluation model will be [GPT-4 Turbo](https://openai.com/gpt-4).

In [4]:
# Share generation args between models
generation_args = {
    "temperature":0,
    "max_tokens":500,
    "stop":"\n\n",
    "model_type":"chat",
    "n": 1
}
# Model specific args
model_info = {
    "gpt-4": {"model": "gpt-4-0125-preview", "api_base": "https://api.openai.com/v1/"},
    "starling": {"model": "Nexusflow/Starling-LM-7B-beta"}
}

In [5]:
# Set up the models
# lm = StarlingLM(**model_info["starling"], **generation_args)
# evaluator_lm = dspy.OpenAI(**model_info["gpt-4"], **generation_args)

lm = dspy.OllamaLocal(model=model_id, base_url='http://localhost:11435')
# evaluator_lm = dspy.OpenAI(**model_info["gpt-4"], **generation_args)
evaluator_lm = dspy.OllamaLocal(model=model_eval_id, base_url='http://localhost:11434')

dspy.configure(lm=lm)

In [6]:
# Testing inference of Starling
lm("What is the capital of Colombia?")

['Bogotá is the capital of Colombia.']

#### Load dataset

The dataset that will be used in this notebook is [gretelai/synthetic_text_to_sql](https://huggingface.co/datasets/gretelai/synthetic_text_to_sql)

In [7]:
# Define random seed
random.seed(1399)

In [8]:
# # Load dataset
# dl = DataLoader()
# trainset = dl.from_huggingface(
#     dataset_name="gretelai/synthetic_text_to_sql", # Dataset name from Huggingface
#     fields=("sql_prompt", "sql_context", "sql"), # Fields needed
#     input_keys=("sql_prompt", "sql_context"), # What our model expects to recieve to generate an output
#     split="train"
# )

# testset = dl.from_huggingface(
#     dataset_name="gretelai/synthetic_text_to_sql", # Dataset name from Huggingface
#     fields=("sql_prompt", "sql_context", "sql"), # Fields needed
#     input_keys=("sql_prompt", "sql_context"), # What our model expects to recieve to generate an output
#     split="test"
# )


# trainset = dl.sample(dataset=trainset, n=100)
# testset = dl.sample(dataset=testset, n=75)
# finetuneset = dl.sample(dataset=trainset, n=100)

# _trainval = dl.train_test_split(dataset=trainset, test_size=0.25, random_state=1399) # 25% of training data for validation
# trainset, valset = _trainval["train"], _trainval["test"]

# len(trainset), len(valset), len(testset)

In [9]:
# Load dataset
dl = DataLoader()

testset = dl.from_huggingface(
    dataset_name="gretelai/synthetic_text_to_sql", # Dataset name from Huggingface
    fields=("sql_prompt", "sql_context", "sql"), # Fields needed
    input_keys=("sql_prompt", "sql_context"), # What our model expects to recieve to generate an output
    split="test"
)
# print(len(testset))

testset = dl.sample(dataset=testset, n=200)

# Calculate the sizes of each set
total_size = len(testset)
train_size = int(total_size * 0.4)  # 40% of the data
val_size = int(total_size * 0.2)  # 20% of the data

# Split the dataset into train, validation and test sets
trainset = testset[:train_size]
valset = testset[train_size:train_size + val_size]
testset = testset[train_size + val_size:]

print(len(trainset), len(valset), len(testset))

80 40 80


In [10]:
# import os
# import json
# # import random
# # from DataLoader import DataLoader  # Assuming DataLoader is defined in a separate module

# # Function to save dataset to a JSON file
# def save_to_json(file_path, data):
#     with open(file_path, 'w') as f:
#         json.dump(data, f)

# # Function to load dataset from a JSON file
# def load_from_json(file_path):
#     with open(file_path, 'r') as f:
#         return json.load(f)

# # Path to the JSON file
# json_file_path = "dataset.json"

# # Check if the JSON file exists
# if os.path.exists(json_file_path):
#     print("Loading dataset from JSON file...")
#     data = load_from_json(json_file_path)
# else:
#     print("Generating dataset and saving to JSON file...")
#     dl = DataLoader()

#     testset = dl.from_huggingface(
#         dataset_name="gretelai/synthetic_text_to_sql",  # Dataset name from Huggingface
#         fields=("sql_prompt", "sql_context", "sql"),  # Fields needed
#         input_keys=("sql_prompt", "sql_context"),  # What our model expects to receive to generate an output
#         split="test"
#     )

#     # Sample the dataset
#     testset = dl.sample(dataset=testset, n=200)

#     # Convert the dataset to a list of dictionaries
#     data = [{"sql_prompt": item["sql_prompt"], "sql_context": item["sql_context"], "sql": item["sql"]} for item in testset]

#     # Save the dataset to a JSON file
#     save_to_json(json_file_path, data)

# # Calculate the sizes of each set
# total_size = len(data)
# train_size = int(total_size * 0.4)  # 40% of the data
# val_size = int(total_size * 0.2)  # 20% of the data

# # Shuffle the data
# random.shuffle(data)

# # Split the dataset into train, validation and test sets
# trainset = data[:train_size]
# valset = data[train_size:train_size + val_size]
# testset = data[train_size + val_size:]

# print(len(trainset), len(valset), len(testset))


In [11]:
# Verify an example of the dataset
sample = trainset[0]
for k, v in sample.items():
    print(f"\n{k.upper()}:\n")
    print(v)


SQL_PROMPT:

What is the average monthly data usage for each mobile network operator?

SQL_CONTEXT:

CREATE TABLE mobile_operators (operator_id INT, operator_name VARCHAR(50)); CREATE TABLE mobile_plans (plan_id INT, plan_name VARCHAR(50), operator_id INT, data_limit INT); CREATE TABLE usage (usage_id INT, subscriber_id INT, plan_id INT, usage_amount INT, usage_date DATE);

SQL:

SELECT o.operator_name, AVG(u.usage_amount) AS avg_monthly_data_usage FROM mobile_operators o INNER JOIN mobile_plans p ON o.operator_id = p.operator_id INNER JOIN usage u ON p.plan_id = u.plan_id WHERE u.usage_date >= DATEADD(month, -1, GETDATE()) GROUP BY o.operator_name;


In [12]:
# # Verify an example of the dataset
# sample = dl.sample(dataset=trainset, n=1)[0]
# for k, v in sample.items():
#     print(f"\n{k.upper()}:\n")
#     print(v)

#### Signature (Input/Output)

In [13]:
class TextToSql(dspy.Signature):
    """Transform a natural language query into a SQL query."""

    sql_prompt = dspy.InputField(desc="Natural language query")
    sql_context = dspy.InputField(desc="Context for the query")
    sql = dspy.OutputField(desc="SQL query")

### Inference

#### Baseline Inference

In [15]:
generate_sql_query = dspy.Predict(signature=TextToSql)

result = generate_sql_query(
    sql_prompt=sample["sql_prompt"],
    sql_context=sample["sql_context"]
)

for k, v in result.items():
    print(f"\n{k.upper()}:\n")
    print(v)

17617691784634347509


AttributeError: 'tuple' object has no attribute 'items'

In [35]:
result

(Prediction(
     sql='SELECT mo.operator_name, AVG(u.usage_amount) as avg_data_usage FROM mobile_operators mo JOIN mobile_plans mp ON mo.operator_id = mp.operator_id JOIN usage u ON mp.plan_id = u.plan_id GROUP BY mo.operator_name;'
 ),
 896761950290501808)

#### ChainOfThought Inference

In [36]:
generate_sql_query = dspy.ChainOfThought(signature=TextToSql)

result = generate_sql_query(
    sql_prompt=sample["sql_prompt"],
    sql_context=sample["sql_context"]
)

# for k, v in result.items():
#     print(f"\n{k.upper()}:\n")
#     print(v)

17617691784634347509


### Metric of evaluation

#### Metric definition

In [37]:
class Correctness(dspy.Signature):
    """Assess if the SQL query accurately answers the given natural language query based on the provided context."""

    sql_prompt = dspy.InputField(desc="Natural language query ")
    sql_context = dspy.InputField(desc="Context for the query")
    sql = dspy.InputField(desc="SQL query")
    correct = dspy.OutputField(desc="Indicate whether the SQL query correctly answers the natural language query based on the given context", prefix="Yes/No:")

In [38]:
eval_correctness_binary = []

def correctness_metric(example, pred, trace=None):
    pred, span_id = pred
    sql_prompt, sql_context, sql = example.sql_prompt, example.sql_context, pred.sql

    correctness = dspy.Predict(Correctness)

    with dspy.context(lm=evaluator_lm): 
        correct = correctness(
            sql_prompt=sql_prompt,
            sql_context=sql_context,
            sql=sql,
        )
    
    score = int(correct.correct=="Yes")

    
    # logging with phoenix
    # Retrieve the span_id from the prediction
    # span_id = getattr(correctness, 'span_id', None)
    # print(correctness)
    # span_id = getattr(correctness, 'span_id', None)
    print(f"Span ID during evaluation: {span_id}")
    
    
    # print(f"Span ID during evaluation: {span_id}")
    # print(type(span_id))
    
    # if span_id is not None:
    #     # convert span_id to hex
    #     span_id = f"{span_id:x}"
    #     print(f"Span ID not None during evaluation: {span_id}")
    #     metrics_correctness_binary = {
    #         'context.span_id': span_id,
    #         'label': correct.correct,
    #         'value': score,
    #         'explanation': "The SQL query correctly answers the natural language query based on the given context."
    #     }
    #     eval_correctness_binary.append(metrics_correctness_binary)
    
    if trace is not None:
        return score == 1
        

    return score

_correctness = correctness_metric(
    example=sample,
    pred=result
)
print(f"Correct SQL query: {'Yes' if _correctness else 'No'}")

7361873453652246119


AttributeError: 'tuple' object has no attribute 'correct'

#### Evaluate single data point

In [ ]:
_correctness = correctness_metric(
    example=sample,
    pred=result
)
print(f"Correct SQL query: {'Yes' if _correctness else 'No'}")

In [ ]:
# get the dataframe from the phoenix
hm = px.Client().get_spans_dataframe(project_name=model_id)

In [ ]:
# look for aaeccafc406d085f in the dataframe
hm[hm['context.span_id'] == 'aaeccafc406d085f']

In [ ]:
evaluator_lm.inspect_history(n=1)

#### Evaluate entire dataset - GPT 3.5

<div style="background-color: #F0F0F0; padding: 10px; border-radius: 5px;"> <p style="color: #4B4B4B; font-size: 18px; font-weight: bold; margin: 0;"> 📊 Baseline Evaluation </p> <p style="color: #4B4B4B; font-size: 16px; margin: 5px 0 0;"> Without any optimization, <strong>Starling7B</strong> achieves an <strong>80% correctness in validation (25 samples)</strong> and <strong>70.07% correctness in test (75 samples).</strong> </p> </div>

In [ ]:
print("GPT 3.5 Turbo - Validation Score: \n")
with dspy.context(lm=dspy.OpenAI(model="gpt-3.5-turbo-0125", api_base="https://api.openai.com/v1/", **generation_args)):
    evaluate = Evaluate(devset=valset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
    evaluate(generate_sql_query)

In [ ]:
print("GPT 3.5 Turbo - Test Score: \n")
with dspy.context(lm=dspy.OpenAI(model="gpt-3.5-turbo-0125", api_base="https://api.openai.com/v1/", **generation_args)):
    evaluate = Evaluate(devset=testset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
    evaluate(generate_sql_query)

#### Evaluate entire dataset - llama3-8b

<div style="background-color: #FFCCCB; padding: 10px; border-radius: 5px;"> <p style="color: #8B0000; font-size: 18px; font-weight: bold; margin: 0;"> ⚠️ Evaluation Stage 1 </p> <p style="color: #8B0000; font-size: 16px; margin: 5px 0 0;"> Without any optimization, <strong>llama3-8b</strong> achieves an <strong>72% correctness in validation (25 samples)</strong> and <strong>50.67% correctness in test (75 samples).</strong> </p> </div>

In [ ]:
print("llama3-8b - Validation Score: \n")
evaluate = Evaluate(devset=valset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(generate_sql_query)

In [ ]:
print("llama3-8b - Test Score: \n")
evaluate = Evaluate(devset=testset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(generate_sql_query)

### Optimize for Text2SQL

#### Create program

In [ ]:
# Define the program ~ You can think of this a Pytorch model.
class TextToSqlProgram(dspy.Module):
    def __init__(self):
        super().__init__()
        # self.program = dspy.ChainOfThought(signature=TextToSql)
    
    def forward(self, sql_prompt, sql_context):
        # context = trace_api.get_current_span()
        current_span = trace_api.get_current_span()
        print(current_span.get_span_context().span_id)
        return self.program(
            sql_prompt=sql_prompt,
            sql_context=sql_context,
            span_id=current_span.get_span_context().span_id
        )

In [ ]:
# write a basic evaluate programm to overwrite the evaluate function, since it does not work with tracking the span_id
class EvaluateWithSpan(dspy.evaluate):
    def __call__(self, model, *args, **kwargs):
        # Check if the model is a module
        if isinstance(model, dspy.Module):
            # Set the model as the program
            self.program = model
        # Call the evaluate function
        return super().__call__(model, *args, **kwargs)

### FewShot

In [ ]:
# Execute the optimizer -> this only adds few shots to the prompt
optimizer = LabeledFewShot(k=4)
optmized_program = optimizer.compile(student=TextToSqlProgram(), trainset=trainset)

In [ ]:
optmized_program(sql_context=sample["sql_context"], sql_prompt=sample["sql_prompt"])

#### What is happening inside?

In [ ]:
lm.inspect_history(n=1)

#### Evaluate the optimized program


<div style="background-color: #FFF8DC; padding: 10px; border-radius: 5px;"> <p style="color: #DAA520; font-size: 18px; font-weight: bold; margin: 0;"> 🌟 Evaluation Stage 2 </p> <p style="color: #DAA520; font-size: 16px; margin: 5px 0 0;"> With <strong>Few Shot</strong> optimization, <strong>llama3-8b</strong> achieves an <strong>64% correctness in validation (25 samples)</strong> and <strong>60% correctness in test (75 samples).</strong> </p> </div>

In [ ]:
print("llama3-8b + FewShotOptimizer - Validation Score: \n")
evaluate = Evaluate(devset=valset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(optmized_program)

In [ ]:
print("llama3-8b + FewShotOptimizer - Test Score: \n")
evaluate = Evaluate(devset=testset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(optmized_program)

### BootstrapFewShotWithRandomSearch

[DSPy docs](https://dspy-docs.vercel.app/docs/building-blocks/optimizers) recommend that in a setup like the one with have at hand, with ~50 samples, the best option is to use `BootstrapFewShotWithRandomSearch`:

![image](assets/dspy.png)

In [ ]:
optimizer2 = BootstrapFewShotWithRandomSearch(metric=correctness_metric, max_bootstrapped_demos=2, num_candidate_programs=8, num_threads=5)
optmized_program_2 = optimizer2.compile(student = TextToSqlProgram(), trainset=trainset, valset=valset)

In [ ]:
optmized_program_2(sql_context=sample["sql_context"], sql_prompt=sample["sql_prompt"])

In [ ]:
lm.inspect_history(n=1)

#### Evaluate the optimized program

<div style="background-color: #E0F8E0; padding: 10px; border-radius: 5px;"> <p style="color: #006400; font-size: 18px; font-weight: bold; margin: 0;"> ✅ Evaluation Stage 3 </p> <p style="color: #006400; font-size: 16px; margin: 5px 0 0;"> With <strong>BootstrapFewShotWithRandomSearch</strong> optimization, <strong>llama3-8b</strong> achieves an <strong>80% correctness in validation (25 samples)</strong> and <strong>68% correctness in test (75 samples).</strong> </p> </div>

In [ ]:
print("llama3-8b + BootstrapFewShotWithRandomSearch - Validation Score: \n")
evaluate = Evaluate(devset=valset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(optmized_program_2)

In [ ]:
print("llama3-8b + BootstrapFewShotWithRandomSearch - Test Score: \n")
evaluate = Evaluate(devset=testset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=0)
evaluate(optmized_program_2)